In [9]:
# SETUP
!pip3 install -r requirements.txt

1414.68s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [1]:
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.utils import ImageReader
from io import BytesIO
import PyPDF2
import os
import yaml
from PIL import Image

# Definimos y usamos la fuente adecuada (basado en el ppt de ejemplo)
pdfmetrics.registerFont(TTFont('Montserrat', 'Montserrat/static/Montserrat-Light.ttf'))

# Fuente con negrita
pdfmetrics.registerFont(TTFont('Montserrat-Bold', 'Montserrat/static/Montserrat-Bold.ttf'))

# Definimos el tamaño 
letter = (1080,1920)

# Leemos el archivo CSV
dataframe = pd.read_csv("input_reciclapp.csv", delimiter=";", dtype={"RUT": str})

# Agrupamos por RUT, Comuna y Tipo de residuo
grouped = dataframe.groupby(['RUT', 'Comuna', 'Mes', 'Nombre cliente', 'Dirección', 'Tipo de residuo'])

# Sumarizamos el peso por cada grupo
summarized_data = grouped['Peso mensual (kg)'].sum().reset_index()


In [2]:
# Función de ayuda para los íconos que sean transparentes
def get_image_with_transparency(image_path):
    """
    Abre una imagen con Pillow y asegura que, si es PNG, tenga transparencia.
    Retorna la imagen como un objeto BytesIO.
    """
    # Abre la imagen
    img = Image.open(image_path)
    
    # Verifica el tipo de imagen
    if image_path.lower().endswith(".png"):
        # Si es PNG, verifica si ya tiene transparencia
        if img.mode != 'RGBA':
            img = img.convert('RGBA')

        # Creamos una imagen de fondo blanca del mismo tamaño que la imagen original
        white_bg = Image.new('RGBA', img.size, (255, 255, 255, 255))
        
        # Pegamos la imagen original sobre el fondo blanco
        white_bg.paste(img, (0, 0), img)  # El último argumento es la máscara de transparencia
        img = white_bg

    # Aquí estamos guardando la imagen modificada en un objeto BytesIO
    img_bytes = BytesIO()
    
    # Guarda la imagen en el formato adecuado según la extensión
    if image_path.lower().endswith(".png"):
        img.save(img_bytes, format="PNG")
    else:
        img.save(img_bytes, format="JPEG")

    img_bytes.seek(0)  # Esto es importante, establece el "archivo" para que se lea desde el principio
    
    return img_bytes

In [3]:
# Función para leer el yaml de configuraciones
def read_configuration(yaml_file):
    with open(yaml_file, 'r') as file:
        conf = yaml.safe_load(file)
    return conf

In [4]:
# Función de escritura de textos pal PDF
def draw_text(c, text, x, y, font_name="Montserrat", font_size=35, bold_phrases=None):
    bold_font = "Montserrat-Bold"
    regular_font = "Montserrat"
    
    # Lista que contendrá los fragmentos de texto
    text_fragments = []

    # Descomponer el texto para que cumpla con la regla de los 50 caracteres
    words = text.split()
    current_line = []
    current_length = 0
    
    for word in words:
        # Si agregar la palabra actual excede el límite, guardamos la línea actual
        if current_length + len(word) > 50:
            text_fragments.append(' '.join(current_line))
            current_length = 0
            current_line = []
        current_line.append(word)
        current_length += len(word) + 1  # +1 por el espacio
        
    # Añadimos la última línea
    text_fragments.append(' '.join(current_line))

    # Dibujar cada fragmento de texto en el canvas, considerando las frases en negrita
    for fragment in text_fragments:
        remaining_text = fragment
        draw_start = x
        while remaining_text:
            is_bold = False
            matched_bold_phrase = None
            for phrase in bold_phrases:
                if remaining_text.startswith(phrase):
                    is_bold = True
                    matched_bold_phrase = phrase
                    break
            if is_bold:
                c.setFont(bold_font, font_size)
                c.drawString(draw_start, y, matched_bold_phrase)
                draw_start += c.stringWidth(matched_bold_phrase, bold_font, font_size)
                remaining_text = remaining_text[len(matched_bold_phrase):]
            else:
                next_space_idx = remaining_text.find(' ')
                if next_space_idx == -1:  # This is the last word
                    word_to_draw = remaining_text
                else:
                    word_to_draw = remaining_text[:next_space_idx]
                c.setFont(regular_font, font_size)
                c.drawString(draw_start, y, word_to_draw)
                draw_start += c.stringWidth(word_to_draw, regular_font, font_size)
                remaining_text = remaining_text[len(word_to_draw):]
            if remaining_text.startswith(' '):  # Remove the space
                draw_start += c.stringWidth(' ', regular_font, font_size)
                remaining_text = remaining_text[1:]
        y -= (font_size + 10)  # Mover a la siguiente línea
    return y

# Función para insertar las imágenes con sus textos
def insert_residue_icons(c, group_data, conf, start_y):
    directory = conf["General_confs"]["Directorio"]
    residue_mapping = conf["Residuos"]
    
    number_of_residues = len(group_data)
    image_width = 134
    image_height = 128
    gap = (1080 - (number_of_residues * image_width)) / (number_of_residues + 1)  # Este es el espacio entre cada imagen
    x = gap  # posición inicial
    
    for _, row in group_data.iterrows():
        residue_type = row['Tipo de residuo']
        residue_weight = row['Peso mensual (kg)']

        # Con esto se intenta escoger la imagen que se represente el residuo
        image_name = residue_mapping.get(residue_type, residue_mapping["Default"])

        # Usamos la función para obtener la imagen con transparencia
        image_path = directory + image_name
        image_obj = get_image_with_transparency(image_path)

        # Usamos ImageReader para convertir el objeto Image de Pillow en algo que ReportLab pueda manejar
        # Art attack (o sea, insertar imagen xD)
        pil_image = ImageReader(image_obj)
        c.drawImage(pil_image, x, start_y, width=image_width, height=image_height)

        c.setFont("Montserrat", 30)
        text_width = c.stringWidth(residue_type, "Montserrat", 28)
        c.drawString(x + (image_width - text_width) / 2, start_y - 40, residue_type)
        
        c.setFont("Montserrat-Bold", 30)
        weight_text = f"{residue_weight}kg"
        text_width = c.stringWidth(weight_text, "Montserrat-Bold", 33)
        c.drawString(x + (image_width - text_width) / 2, start_y - 80, weight_text)

        x += image_width + gap

    return start_y - 180 - 80  # Esto messirve para ir ajustando las alturas del resto de imágenes


In [5]:
# función mejorada de dibujo de textos con estilos
def create_text_pdf_conestilo(group_data, output_filename):
    c = canvas.Canvas(output_filename, pagesize=(1080,1920))
    c.setFont("Montserrat", 35)
    width, height = (1080,1920)

    # Definimos la lista de frases que deben ir en negrita
    bold_phrases = [
        "Reciclapp SpA",
        group_data['Nombre cliente'].iloc[0].strip().title(),
        group_data['Mes'].iloc[0]
    ]
    
    # Creamos el texto principal
    main_text = (f"Reciclapp SpA, RUT 76.596.736-8 certifica que en alianza con la empresa {group_data['Nombre cliente'].iloc[0].strip().title()} "
                 f"RUT {group_data['RUT'].iloc[0]}, ubicada en {group_data['Dirección'].iloc[0]}, "
                 f"{group_data['Comuna'].iloc[0]}, ha reciclado junto a nosotros durante el mes de "
                 f"{group_data['Mes'].iloc[0]} un total de:")

    y_position = draw_text(c, main_text, 110, 1200, bold_phrases=bold_phrases)
    y_position = insert_residue_icons(c, group_data, read_configuration('conf.yaml'), y_position - 150)

    c.save()



In [6]:
# Definimos la función que hacer el overlay
def overlay_pdfs(input_pdf, overlay_pdf, output_pdf):
    with open(input_pdf, "rb") as main_file, open(overlay_pdf, "rb") as overlay_file, open(output_pdf, "wb") as output_file:
        # Leer los PDFs
        main_pdf = PyPDF2.PdfFileReader(main_file)
        overlay_pdf = PyPDF2.PdfReader(overlay_file)

        # Crear un PDF de salida
        pdf_writer = PyPDF2.PdfFileWriter()

        # Asegurarse de que ambos PDFs tengan la misma cantidad de páginas
        num_pages = min(main_pdf.getNumPages(), overlay_pdf.getNumPages())

        for i in range(num_pages):
            # Combinar las páginas
            page = main_pdf.getPage(i)
            page.mergePage(overlay_pdf.getPage(i))
            pdf_writer.addPage(page)

        # Guardar el PDF combinado
        pdf_writer.write(output_file)

In [7]:
# Generamos un PDF para cada RUT y Comuna
unique_groups = summarized_data.groupby(['RUT', 'Comuna'])
for name, group in unique_groups:
    nombre_cliente = group["Nombre cliente"].iloc[0].strip().replace(" ","-") # esto nos permite acceder al nombre del cliente, quitarle espacios finales y reemplazar espacios del medio por guiones
    output_filename = f"tmp_{nombre_cliente}_{name[1]}.pdf"
    print(output_filename.replace("tmp_",""))
    create_text_pdf_conestilo(group, output_filename)
    overlay_pdfs("maqueta.pdf", output_filename, "Resultados_PDF/" + output_filename.replace("tmp_",""))
    os.remove(output_filename)
    

Club-de-Planeadores_VITACURA.pdf
Confuturo_CONCEPCION.pdf
Confuturo_SANTIAGO.pdf
Confuturo_VIÑA DEL MAR.pdf
Compass-Group_SANTIAGO.pdf
Dark-Kitchen-Hood_PROVIDENCIA.pdf
Awto_PROVIDENCIA.pdf
Adidas-corporativo_VITACURA.pdf
Autofact_PROVIDENCIA.pdf


In [8]:
# cleanup
# os.system("rm Resultados_PDF/*")